In [21]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [22]:
house_dict = []

for page in range(1, 6):
    linked = 'https://www.zillow.com/ca/condos/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-153.847663625,"east":-84.765632375,"south":12.777434491046959,"north":56.013075839104054},"mapZoom":4,"regionSelection":[{"regionId":9,"regionType":2}],"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false}},"isListVisible":true,"isMapVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw")[1].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 bloUvX")[2].text
                
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/homedetails/930-Via-Mil-Cumbres-UNIT-27-Solana-Beach-CA-92075/16726910_zpid/
zestimate $599,008
rent zestimate $3,000/mo
https://www.zillow.com/homedetails/23839-Del-Monte-Dr-UNIT-88-Santa-Clarita-CA-91355/20227478_zpid/
zestimate $456,003
rent zestimate $2,325/mo
https://www.zillow.com/homedetails/1309-W-Mission-Blvd-U-6-Ontario-CA-91762/69012056_zpid/
zestimate $286,782
rent zestimate $1,750/mo
https://www.zillow.com/homedetails/5350-White-Oak-Ave-APT-101-Los-Angeles-CA-91316/19978364_zpid/
zestimate $563,973
rent zestimate $2,850/mo
https://www.zillow.com/homedetails/317-Streamwood-Irvine-CA-92620/25720167_zpid/
zestimate $400,900
rent zestimate $2,000/mo
https://www.zillow.com/homedetails/5075-La-Cuenta-Dr-San-Diego-CA-92124/52517684_zpid/
zestimate $579,951
rent zestimate $2,695/mo
https://www.zillow.com/homedetails/640-Seabright-Cir-Costa-Mesa-CA-92627/145646647_zpid/
zestimate $742,901
rent zestimate $3,300/mo
https://www.zillow.com/homedetails/1120-N-El-C

zestimate $631,056
rent zestimate $2,850/mo
https://www.zillow.com/homedetails/3747-Radburn-Dr-South-San-Francisco-CA-94080/2076874991_zpid/
zestimate $859,475
rent zestimate null
https://www.zillow.com/homedetails/4346-52nd-St-UNIT-7-San-Diego-CA-92115/70597460_zpid/
zestimate $210,000
rent zestimate $1,645/mo
https://www.zillow.com/homedetails/5508-Saxon-Way-Riverbank-CA-95367/61342168_zpid/
zestimate $495,004
rent zestimate $2,395/mo
https://www.zillow.com/homedetails/967-E-Mason-Ln-96-Anaheim-CA-92805/325863610_zpid/
zestimate $651,555
rent zestimate null
https://www.zillow.com/homedetails/6122-Westport-Ln-Citrus-Heights-CA-95621/94692430_zpid/
zestimate $302,141
rent zestimate $1,750/mo
https://www.zillow.com/homedetails/322-W-Macdonald-Ave-Richmond-CA-94801/18557211_zpid/
zestimate $283,352
rent zestimate $2,650/mo
https://www.zillow.com/homedetails/445-N-Sierra-Bonita-Ave-Los-Angeles-CA-90036/2076765590_zpid/
zestimate $459,362
rent zestimate null
https://www.zillow.com/homedeta

zestimate $241,164
rent zestimate null
https://www.zillow.com/homedetails/25810-Van-Leuven-St-Loma-Linda-CA-92354/17371076_zpid/
zestimate $300,000
rent zestimate $1,700/mo
https://www.zillow.com/homedetails/4633-Smoke-River-Ct-San-Jose-CA-95136/19702068_zpid/
zestimate $705,775
rent zestimate $3,200/mo
https://www.zillow.com/homedetails/7675-Roberta-Ln-Antelope-CA-95843/25964280_zpid/
zestimate $282,615
rent zestimate $1,795/mo
https://www.zillow.com/homedetails/1111-S-Grand-Ave-APT-1101-Los-Angeles-CA-90015/79796246_zpid/
zestimate $1,394,637
rent zestimate $5,254/mo
https://www.zillow.com/homedetails/12218-Orchid-Ln-APT-C-Moreno-Valley-CA-92557/18005813_zpid/
zestimate $129,000
rent zestimate $1,810/mo
https://www.zillow.com/homedetails/4577-Villas-Dr-Bonita-CA-91902/79974308_zpid/
zestimate $500,919
rent zestimate $2,800/mo
https://www.zillow.com/homedetails/2101-E-21st-St-UNIT-303-Signal-Hill-CA-90755/21198524_zpid/
zestimate $441,046
rent zestimate $2,450/mo
https://www.zillow.co

zestimate $600,053
rent zestimate $2,600/mo
https://www.zillow.com/homedetails/501-Gibson-Dr-APT-1711-Roseville-CA-95678/63055547_zpid/
zestimate $305,601
rent zestimate $1,695/mo
https://www.zillow.com/homedetails/490-Anderwood-Ct-APT-3-Pomona-CA-91768/21520550_zpid/
zestimate $357,910
rent zestimate $2,100/mo
https://www.zillow.com/homedetails/453-Sunningdale-Dr-Rancho-Mirage-CA-92270/17743841_zpid/
zestimate $275,000
rent zestimate $2,400/mo
https://www.zillow.com/homedetails/5585-E-Pacific-Coast-Hwy-UNIT-145-Long-Beach-CA-90804/21213468_zpid/
zestimate $295,842
rent zestimate $1,775/mo
https://www.zillow.com/homedetails/16377-Lakeshore-Dr-UNIT-6I-Lake-Elsinore-CA-92530/69293755_zpid/
zestimate $199,001
rent zestimate $1,650/mo
https://www.zillow.com/homedetails/15420-Jack-Ct-Lathrop-CA-95330/15346110_zpid/
zestimate $502,142
rent zestimate $2,395/mo
https://www.zillow.com/homedetails/661-Cherry-Ave-Sonoma-CA-95476/15798612_zpid/
zestimate $460,755
rent zestimate $2,700/mo
https://w

In [23]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,Condo for sale,"$599,900","930 Via Mil Cumbres UNIT 27, Solana Beach, CA ...",930 Via Mil Cumbres UNIT 27,SolanaBeach,CA,92075,3,2,"1,242",null,null,Condo,1979,"Carport, Covered",$415/mo,"$599,008","$3,000/mo",$483,https://www.zillow.com/homedetails/930-Via-Mil...
1,Condo for sale,"$451,100","23839 Del Monte Dr UNIT 88, Santa Clarita, CA ...",23839 Del Monte Dr UNIT 88,SantaClarita,CA,91355,2,2,"1,067",null,null,Condo,1987,"Garage - Detached, Covered",$285/mo,"$456,003","$2,325/mo",$423,https://www.zillow.com/homedetails/23839-Del-M...
2,Condo for sale,"$279,900","1309 W Mission Blvd #U-6, Ontario, CA 91762",1309 W Mission Blvd #U-6,Ontario,CA,91762,2,2,845,null,null,Condo,1988,"Garage, Carport, Covered",$310/mo,"$286,782","$1,750/mo",$331,https://www.zillow.com/homedetails/1309-W-Miss...
3,Condo for sale,"$559,900","5350 White Oak Ave APT 101, Los Angeles, CA 91316",5350 White Oak Ave APT 101,LosAngeles,CA,91316,2,2,"1,452",null,null,Condo,1990,"Carport, Covered",$535/mo,"$563,973","$2,850/mo",$386,https://www.zillow.com/homedetails/5350-White-...
4,Condo for sale,"$400,900","317 Streamwood, Irvine, CA 92620",317 Streamwood,Irvine,CA,92620,2,1,809,null,null,Condo,1977,"Carport, Covered",$295/mo,"$400,900","$2,000/mo",$496,https://www.zillow.com/homedetails/317-Streamw...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,,"$369,900","302 Arundell Cir, Fillmore, CA 93015",302 Arundell Cir,Fillmore,CA,93015,3,2,"1,259",null,null,Condo,2004,"Garage, Covered",$370/mo,"$373,299","$2,200/mo",$294,https://www.zillow.com/homedetails/302-Arundel...
196,,"$649,888","1923 Palomino Dr, West Covina, CA 91791",1923 Palomino Dr,WestCovina,CA,91791,4,3,"2,082",null,null,Condo,1997,"Garage - Attached, Covered",$240/mo,"$658,328",null,$312,https://www.zillow.com/homedetails/1923-Palomi...
197,,"$545,000","39 Bayley St, Mission Viejo, CA 92694",39 Bayley St,MissionViejo,CA,92694,2,3,"1,583",null,null,Condo,2003,"Garage, Garage - Attached, Covered",$193/mo,"$564,796","$2,800/mo",$344,https://www.zillow.com/homedetails/39-Bayley-S...
198,,"$345,000","233 Sandy Beach Rd, Vallejo, CA 94590",233 Sandy Beach Rd,Vallejo,CA,94590,2,2,"1,018",null,null,Condo,1992,"Garage, Garage - Attached, Garage - Detached, ...",$307/mo,"$348,670",null,$339,https://www.zillow.com/homedetails/233-Sandy-B...


In [24]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q4-2020_Condo_Data_page1-5.csv", index=False, header=True)